In [1]:
import pandas as pd
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import InvalidSessionIdException
from bs4 import BeautifulSoup


In [3]:
def book(file_path):
    # 엑셀
    df = pd.read_csv(file_path)

    # ItemId로 상세 URL 생성
    de_urls = df["ItemId"].astype(str).tolist()
    detail_urls_path = [
        "https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=" + item_id
        for item_id in de_urls
    ]

    # 크롤링 준비
    reviews_book = []
    valid_isbns = []

    driver = webdriver.Chrome()

    for i, link in enumerate(detail_urls_path):
        try:
            driver.get(link)
            time.sleep(random.uniform(2, 4))

            # 리뷰 100개 보기 클릭
            try:
                reviews_100_btn = driver.find_element(By.CSS_SELECTOR, "div.info_list a:nth-of-type(3)")
                reviews_100_btn.click()
                time.sleep(random.uniform(1, 1.5))
            except:
                print(f"[{i}] {link} - 리뷰 100개 버튼 없음, 건너뜀")
                continue

            # 전체 탭 클릭
            try:
                total_btn = driver.find_element(By.CSS_SELECTOR, "div.Btitle_box a#tabTotal")
                driver.execute_script("arguments[0].click();", total_btn)
                time.sleep(random.uniform(1, 1.5))
            except:
                print(f"[{i}] {link} - 전체 리뷰 탭 없음, 건너뜀")
                continue

            # 더보기 반복 클릭
            while True:
                try:
                    more_btn = driver.find_element(By.CSS_SELECTOR, "div#CommentReviewList div.review_number div.Ere_btn_more a")
                    more_btn.click()
                    time.sleep(random.uniform(2.5, 3.5))
                except:
                    print(f"[{i}] {link} - 더보기 버튼 없음 또는 클릭 완료")
                    break

            soup = BeautifulSoup(driver.page_source, "html.parser")
            nickname_tags = soup.select("div#CommentReviewList div.hundred_list div.left a:nth-of-type(1)")
            star_blocks = soup.select("div#CommentReviewList div.hundred_list div.HL_star")
            review_tags = soup.select("div#CommentReviewList div.hundred_list div.blog_list3 li:nth-of-type(1) span:nth-of-type(2)")

            book_reviews = []
            for nickname_tag, star_tag, review_tag in zip(nickname_tags, star_blocks, review_tags):
                nickname = nickname_tag.get_text(strip=True)
                star_imgs = star_tag.find_all("img", src=lambda x: x and "icon_star_on" in x)
                rating = len(star_imgs)
                review = review_tag.get_text(strip=True)
                book_reviews.append((nickname, rating, review))

            # 리뷰가 있을 때만 저장
            if book_reviews and not pd.isna(df["ISBN13"].iloc[i]):
                reviews_book.append(book_reviews)
                valid_isbns.append(df["ISBN13"].iloc[i])

        except InvalidSessionIdException:
            print(f"[{i}] {link} - 세션이 만료되었습니다. 드라이버 재시작 중...")
            driver = webdriver.Chrome()
            driver.get(link)
            time.sleep(random.uniform(2, 4))
            continue

        except Exception as e:
            print(f"에러 @ {link}: {e}")
            continue

    driver.quit()

    # 결과 정리 및 저장
    rows = []
    for book_reviews, isbn in zip(reviews_book, valid_isbns):
        for user_id, rating, review in book_reviews:
            rows.append({
                'user_id': user_id,
                'isbn': isbn,
                'rating': rating,
                'review': review
            })

    books = pd.DataFrame(rows)
    books.to_csv("2001_주말.csv", index=False)

    return books


In [7]:
book("./data/역대_2001.csv")

[0] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=59172854  - 더보기 버튼 없음 또는 클릭 완료
[1] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=229734  - 더보기 버튼 없음 또는 클릭 완료
[2] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=350924171  - 전체 리뷰 탭 없음, 건너뜀
[3] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=11277527  - 더보기 버튼 없음 또는 클릭 완료
[4] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=33471  - 더보기 버튼 없음 또는 클릭 완료
[5] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=85100618  - 전체 리뷰 탭 없음, 건너뜀
[6] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=351448761  - 전체 리뷰 탭 없음, 건너뜀
[7] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=351447845  - 전체 리뷰 탭 없음, 건너뜀
[8] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=134925661  - 더보기 버튼 없음 또는 클릭 완료
[9] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=351447950  - 전체 리뷰 탭 없음, 건너뜀
[10] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=270976  - 더보기 버튼 없음 또는 클릭 완료
[11] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=351448100  - 전

,user_id,isbn,rating,review
0,th-won,9.788980e+12,5,"참 좋다. 이 책 참 좋다. 많지 않은 내용이지만, 읽고 또 읽게 만드는 작품"
1,주한길,9.788980e+12,3,˝이제는 변하지 않으면 살아남을 수 없다.두려움을 없앤다면 성공의 길은 반드시 열린...
2,사과향기,9.788980e+12,3,글쎄...보는 관점에 따라 다를수도 있지만 현대의 다양성에는 다소 못미치는 책입니다...
3,예향지기,9.788980e+12,5,늘 초심을 일깨우게 해주는 책^^다시 치즈를 찾아 일어날 시간이당~~
4,psy_elsa,9.788980e+12,5,https://youtu.be/gMGHKbkyR1g이 책읽고 정말 변화에 대해 긴장...
...,...,...,...,...
4165,ysky333,9.788932e+12,4,우리 아이가 너무 재미있게 읽더군요 ~ 그리고 자기도 조커 하나 만들어달라고...
4166,치즈찌,9.788932e+12,4,초등판 ‘죽은시인의 사회‘같네요 아직 초3인 아들이 그‘단어 ㅅㅅ‘를 어떻게 받아 ...
4167,alfacefa,9.788932e+12,5,완전좋아요
4168,최유빈,9.788932e+12,5,"마음이 따뜻해지는 소설. 아이도, 어른도 읽기 좋은 이야기"


In [23]:
def book(file_path):
    # 엑셀
    df = pd.read_csv(file_path)

    # ItemId로 상세 URL 생성
    de_urls = df["ItemId"].astype(str).tolist()
    detail_urls_path = [
        "https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=" + item_id
        for item_id in de_urls
    ]

    # 크롤링 준비
    reviews_book = []
    valid_isbns = []

    driver = webdriver.Chrome()

    for i, link in enumerate(detail_urls_path):
        try:
            driver.get(link)
            time.sleep(random.uniform(2, 4))

            # 리뷰 100개 보기 클릭
            try:
                reviews_100_btn = driver.find_element(By.CSS_SELECTOR, "div.info_list a:nth-of-type(3)")
                reviews_100_btn.click()
                time.sleep(random.uniform(1, 1.5))
            except:
                print(f"[{i}] {link} - 리뷰 100개 버튼 없음, 건너뜀")
                continue

            # 전체 탭 클릭
            try:
                total_btn = driver.find_element(By.CSS_SELECTOR, "div.Btitle_box a#tabTotal")
                driver.execute_script("arguments[0].click();", total_btn)
                time.sleep(random.uniform(1, 1.5))
            except:
                print(f"[{i}] {link} - 전체 리뷰 탭 없음, 건너뜀")
                continue

            # 더보기 반복 클릭
            while True:
                try:
                    more_btn = driver.find_element(By.CSS_SELECTOR, "div#CommentReviewList div.review_number div.Ere_btn_more a")
                    more_btn.click()
                    time.sleep(random.uniform(2.5, 3.5))
                except:
                    print(f"[{i}] {link} - 더보기 버튼 없음 또는 클릭 완료")
                    break

            soup = BeautifulSoup(driver.page_source, "html.parser")
            nickname_tags = soup.select("div#CommentReviewList div.hundred_list div.left a:nth-of-type(1)")
            star_blocks = soup.select("div#CommentReviewList div.hundred_list div.HL_star")
            review_tags = soup.select("div#CommentReviewList div.hundred_list div.blog_list3 li:nth-of-type(1) span:nth-of-type(2)")

            book_reviews = []
            for nickname_tag, star_tag, review_tag in zip(nickname_tags, star_blocks, review_tags):
                nickname = nickname_tag.get_text(strip=True)
                star_imgs = star_tag.find_all("img", src=lambda x: x and "icon_star_on" in x)
                rating = len(star_imgs)
                review = review_tag.get_text(strip=True)
                book_reviews.append((nickname, rating, review))

            # 리뷰가 있을 때만 저장
            if book_reviews and not pd.isna(df["ISBN13"].iloc[i]):
                reviews_book.append(book_reviews)
                valid_isbns.append(df["ISBN13"].iloc[i])

        except InvalidSessionIdException:
            print(f"[{i}] {link} - 세션이 만료되었습니다. 드라이버 재시작 중...")
            driver = webdriver.Chrome()
            driver.get(link)
            time.sleep(random.uniform(2, 4))
            continue

        except Exception as e:
            print(f"에러 @ {link}: {e}")
            continue

    driver.quit()

    # 결과 정리 및 저장
    rows = []
    for book_reviews, isbn in zip(reviews_book, valid_isbns):
        for user_id, rating, review in book_reviews:
            rows.append({
                'user_id': user_id,
                'isbn': isbn,
                'rating': rating,
                'review': review
            })

    books = pd.DataFrame(rows)
    books.to_csv("2002_주말.csv", index=False)

    return books


In [25]:
book("./data/역대_2002.csv")

[0] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=229734  - 더보기 버튼 없음 또는 클릭 완료
[1] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=2553595  - 더보기 버튼 없음 또는 클릭 완료
[2] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=240121933  - 더보기 버튼 없음 또는 클릭 완료
[3] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=195834968  - 더보기 버튼 없음 또는 클릭 완료
[4] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=59172854  - 더보기 버튼 없음 또는 클릭 완료
[5] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=304191  - 더보기 버튼 없음 또는 클릭 완료
[6] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=33471  - 더보기 버튼 없음 또는 클릭 완료
[7] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=350924843  - 전체 리뷰 탭 없음, 건너뜀
[8] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=25759081  - 더보기 버튼 없음 또는 클릭 완료
[9] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=6666531  - 더보기 버튼 없음 또는 클릭 완료
[10] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=108277974  - 더보기 버튼 없음 또는 클릭 완료
[11] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=24419

,user_id,isbn,rating,review
0,마늘코,9.788995e+12,5,"신랑이 읽어주다가 목소리떨리며 눈물을 훔치더라구요..저도 뭉클해서 눈물이 흐르고,,..."
1,ncdr,9.788995e+12,5,읽을 때마다 눈물이 흘러서 아이들에게 읽어주기가 힘든 책...
2,빵은,9.788995e+12,5,"내용이 완전 감동적임.. 자라고, 자라고, 자라서.. 이부분을 아이가 너무 좋아하네요."
3,후애(厚愛),9.788995e+12,5,반값도서~ 탐이 나는 그림책~
4,ipromiseu,9.788995e+12,5,엄마를 위한 책...다 읽고는 마음이 짠...
...,...,...,...,...
5360,오늘도 맑음,9.788973e+12,3,그저 그런 감동•••••
5361,하현순,9.788973e+12,5,사람내음이 나는 따뜻하고 아름다운 책입니다.가볍게 읽기에 얇고 그림동화라 누구나 추...
5362,다인이엄마,9.788973e+12,5,우동한그릇의 작지만 따뜻한 마음들이 모이면 더 아름다운 세상으로 바꾸겠구나하는 생각...
5363,不二,9.788972e+12,5,"전에 한 번 읽었던 것 같은데, 지금 다시 보니 엄청 깊은 이야기를 담고 있다. 내..."


In [26]:
def book(file_path):
    # 엑셀
    df = pd.read_csv(file_path)

    # ItemId로 상세 URL 생성
    de_urls = df["ItemId"].astype(str).tolist()
    detail_urls_path = [
        "https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=" + item_id
        for item_id in de_urls
    ]

    # 크롤링 준비
    reviews_book = []
    valid_isbns = []

    driver = webdriver.Chrome()

    for i, link in enumerate(detail_urls_path):
        try:
            driver.get(link)
            time.sleep(random.uniform(2, 4))

            # 리뷰 100개 보기 클릭
            try:
                reviews_100_btn = driver.find_element(By.CSS_SELECTOR, "div.info_list a:nth-of-type(3)")
                reviews_100_btn.click()
                time.sleep(random.uniform(1, 1.5))
            except:
                print(f"[{i}] {link} - 리뷰 100개 버튼 없음, 건너뜀")
                continue

            # 전체 탭 클릭
            try:
                total_btn = driver.find_element(By.CSS_SELECTOR, "div.Btitle_box a#tabTotal")
                driver.execute_script("arguments[0].click();", total_btn)
                time.sleep(random.uniform(1, 1.5))
            except:
                print(f"[{i}] {link} - 전체 리뷰 탭 없음, 건너뜀")
                continue

            # 더보기 반복 클릭
            while True:
                try:
                    more_btn = driver.find_element(By.CSS_SELECTOR, "div#CommentReviewList div.review_number div.Ere_btn_more a")
                    more_btn.click()
                    time.sleep(random.uniform(2.5, 3.5))
                except:
                    print(f"[{i}] {link} - 더보기 버튼 없음 또는 클릭 완료")
                    break

            soup = BeautifulSoup(driver.page_source, "html.parser")
            nickname_tags = soup.select("div#CommentReviewList div.hundred_list div.left a:nth-of-type(1)")
            star_blocks = soup.select("div#CommentReviewList div.hundred_list div.HL_star")
            review_tags = soup.select("div#CommentReviewList div.hundred_list div.blog_list3 li:nth-of-type(1) span:nth-of-type(2)")

            book_reviews = []
            for nickname_tag, star_tag, review_tag in zip(nickname_tags, star_blocks, review_tags):
                nickname = nickname_tag.get_text(strip=True)
                star_imgs = star_tag.find_all("img", src=lambda x: x and "icon_star_on" in x)
                rating = len(star_imgs)
                review = review_tag.get_text(strip=True)
                book_reviews.append((nickname, rating, review))

            # 리뷰가 있을 때만 저장
            if book_reviews and not pd.isna(df["ISBN13"].iloc[i]):
                reviews_book.append(book_reviews)
                valid_isbns.append(df["ISBN13"].iloc[i])

        except InvalidSessionIdException:
            print(f"[{i}] {link} - 세션이 만료되었습니다. 드라이버 재시작 중...")
            driver = webdriver.Chrome()
            driver.get(link)
            time.sleep(random.uniform(2, 4))
            continue

        except Exception as e:
            print(f"에러 @ {link}: {e}")
            continue

    driver.quit()

    # 결과 정리 및 저장
    rows = []
    for book_reviews, isbn in zip(reviews_book, valid_isbns):
        for user_id, rating, review in book_reviews:
            rows.append({
                'user_id': user_id,
                'isbn': isbn,
                'rating': rating,
                'review': review
            })

    books = pd.DataFrame(rows)
    books.to_csv("2003_주말.csv", index=False)

    return books


In [27]:
book("./data/역대_2003.csv")

[0] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=1951458  - 더보기 버튼 없음 또는 클릭 완료
[1] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=19632807  - 더보기 버튼 없음 또는 클릭 완료
[2] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=229734  - 더보기 버튼 없음 또는 클릭 완료
[3] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=395888  - 더보기 버튼 없음 또는 클릭 완료
[4] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=519125  - 더보기 버튼 없음 또는 클릭 완료
[5] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=404023  - 더보기 버튼 없음 또는 클릭 완료
[6] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=4855  - 더보기 버튼 없음 또는 클릭 완료
[7] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=25759081  - 더보기 버튼 없음 또는 클릭 완료
[8] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=33471  - 더보기 버튼 없음 또는 클릭 완료
[9] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=360910  - 더보기 버튼 없음 또는 클릭 완료
[10] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=270976  - 더보기 버튼 없음 또는 클릭 완료
[11] https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=195834968  - 더보기 

,user_id,isbn,rating,review
0,코레자와,9.788933e+12,5,그저 최고라고밖엔 다른말이필요없는 천재작가의 야심작
1,Lunot,9.788933e+12,2,"소재는 독특하지만, 이야기들은 어설프다."
2,가비,9.788933e+12,5,이 묘한 느낌은 뭔지.. 이런 세상이 올까 두렵습니다.. 간만에 좋은 책 득템!!
3,mumulmu,9.788933e+12,5,"초등학교 고학년부터 중고등학생, 일반인까지 그 상상력에 감탄하며 읽는 책."
4,상하이슈슈,9.788933e+12,4,베르나르 베르베르의 소설 중 ｀뇌｀는 재미 있게 읽어 나무를 읽어보니 역시 베르나르...
...,...,...,...,...
6250,행운유수,9.788996e+12,5,노동운돔의 개념을 전국민에게 충격적으로각인시킨 선각자가 전태일 열사입니다
6251,홍준영,9.788996e+12,5,매우 중요한 책입니다.
6252,이현미,9.788996e+12,5,한번쯤 읽어보세요추천합니다
6253,지나가던,9.788996e+12,5,두 가지 의미에서 ˝어떻게 이렇게 살 수 있는가˝라는 질문을 던지게 된다. 사람이 ...


In [11]:
df = pd.read_csv("역대2022_일요일_드라이버문제.csv")

In [13]:
pd.options.display.float_format = '{:.1f}'.format

In [15]:
len(df["isbn"].unique())

282